In [1]:
# 시군구 코드 매핑 파일 만들기

import pandas as pd
import _config
import _codes
import _util

file_nm = '05_22_school'
df = pd.read_csv(_config.school_read_path + '\\' + file_nm + '.csv', encoding=_config.euc_kr)
drop_value = '면적'
indexes_to_drop = df[df['항목'].str.contains('면적')].index
df = df.drop(indexes_to_drop)

#row to column
df = pd.melt(
    df
    , id_vars=['소재지(시군구)별', '교육과정별', '항목', '단위']
    , value_vars=[f'{year} 년' for year in range(2005, 2023)]
    , var_name='year'
    , value_name='값'
    , ignore_index=True
)
# df = df.dropna(subset=['year'])
new_data_list = []
for idx, row in df.iterrows():
    if row['값'] == 0: continue
    if pd.isna(row['값']): continue

    new_data_list.append(row)
df = pd.DataFrame(new_data_list)

# df = df.dropna(subset=['값'])
df.reset_index(drop=True, inplace=True)

sido_cd = None
for idx, row in df.iterrows():
    sgg = row['소재지(시군구)별']

    #sido_cd setting
    sido_cd = _util.sido_cd_map.get(sgg, sido_cd)
    df.loc[idx, 'sido_cd'] = sido_cd

    #year
    year = str(row['year']).replace('년', '')
    year = int(year.strip())

    if year < 2005: sido_sgg_cd_map = _util.process_sgg_codes(_codes.sgg_codes_2000, 1)
    elif year < 2010: sido_sgg_cd_map = _util.process_sgg_codes(_codes.sgg_codes_2005, 1)
    elif year < 2015: sido_sgg_cd_map = _util.process_sgg_codes(_codes.sgg_codes_2010, 1)
    elif year < 2020: sido_sgg_cd_map = _util.process_sgg_codes(_codes.sgg_codes_2015, 1)
    elif year < 2023: sido_sgg_cd_map = _util.process_sgg_codes(_codes.sgg_codes_2020, 1)
    elif year >= 2023: sido_sgg_cd_map = _util.process_sgg_codes(_codes.sgg_codes_2023, 1)
    else: 
        # print(year)
        raise Exception
    
    # year 4words setting
    df.loc[idx, 'year'] = f'`{year}'

    # 시도 코드 매핑
    sido_cd = _util.sido_cd_map.get(sgg, sido_cd)
    df.loc[idx, 'sido_cd'] = f'`{str(sido_cd)}'  # 시도 코드 할당

    # 시군구 코드 매핑2
    sido = sido_sgg_cd_map.get(sido_cd, {})
    sido = dict(sido)
    sgg_cd = sido.get(sgg, None)

    if sgg == '전라북도': sgg_cd = '35'
    df.loc[idx, 'sgg_cd'] = f'`{str(sgg_cd)}'

#row to column
df = df.pivot_table(index=['sido_cd', 'sgg_cd', '소재지(시군구)별', 'year', '항목', '단위'], 
    columns='교육과정별', 
    values='값',
    aggfunc='sum',
    fill_value=0
)  # 여기서 aggfunc는 데이터 집계 방법을 정의합니다.
df = df.reset_index()

_cols = {}
for col in df.columns:
    val = col
    if val.count('시군구') > 0: val = 'sgg_nm'
    _cols[col] = val
df.rename(columns=_cols, inplace=True)

df = df.drop(columns='각종학교', errors='raise')
column_list = [
    'sido_cd', 'sgg_cd', 'sgg_nm', 'year'
    , '유치원', '초등학교', '중학교', '고등학교', '특수학교'
]
#column sort
df = df[column_list]

df.to_csv(_config.school_write_path + '\\' + file_nm + '.csv', encoding=_config.euc_kr, index=False)

In [10]:
# db create table && data insert

import pandas as pd
import _util
import _config
import _codes

files = _util.get_files(_config.school_write_path)
files = [file for file in files if file.count('_school')]

df_list = []
for file in files:
    df_list.append(pd.read_csv(_config.school_write_path + '\\' + file, encoding=_config.euc_kr))

df = pd.concat(df_list, ignore_index=False)
cols = list(df.columns)

if len(cols) == 0: exit()

_cols = []
for col in cols:
    col = f'"{col}"'

    if col.count('sido_cd') > 0: val = f'{col} varchar(2)'
    elif col.count('sgg_cd'): val = f'{col} varchar(5)'
    elif col.count('sgg_nm'): val = f'{col} varchar(20)'
    elif col.count('year'): val = f'{col} varchar(4)'
    elif col.count('항목'): val = f'{col} varchar(10)'
    else: val = f'{col} float8'
    _cols.append(val)
sql = f'''
    do $$
        begin
        if exists (select 1 from pg_tables where tablename = 'school') then
        drop table school cascade;
        end if;
        if not exists (select 1 from pg_tables where tablename = 'school') then
        create table school(
            {', '.join(_cols)}
        );
        end if;
    end $$;
'''

_util.execute_sql(sql)

def insert_data(df, columns):
    #column setting
    columns = list(columns)
    for idx in range(len(columns)):
        columns[idx] = f'"{columns[idx]}"'

    #to use dataframe for clause
    data = df.values.tolist()
    for chunk in _util.chunker(data, 1000):
        values = []
        for row in chunk:
            for idx in range(len(columns)):
                if str(row[idx]).count('`') > 0:
                    val = str(row[idx]).replace('`', '')
                    row[idx] = f'\'{val}\''
                else:
                    row[idx] = f'\'{str(row[idx])}\''
                    
                if row[idx] == 'None':
                    row[idx] = '\'\''

            values.append(f"({', '.join(row)})")

        sql = f'''
            insert into school(
                {','.join(columns)})
            values {', '.join(values)}
        '''
        _util.execute_sql(sql)

    
insert_data(df, df.columns)





    do $$
        begin
        if exists (select 1 from pg_tables where tablename = 'school') then
        drop table school cascade;
        end if;
        if not exists (select 1 from pg_tables where tablename = 'school') then
        create table school(
            "sido_cd" varchar(2), "sgg_cd" varchar(5), "sgg_nm" varchar(20), "year" varchar(4), "유치원" float8, "초등학교" float8, "중학교" float8, "고등학교" float8, "특수학교" float8
        );
        end if;
    end $$;


            insert into school(
                "sido_cd","sgg_cd","sgg_nm","year","유치원","초등학교","중학교","고등학교","특수학교")
            values ('00', 'None', '전국', '2005', '532', '4604', '2252', '2029', '90'), ('00', 'None', '전국', '2006', '505', '4676', '2284', '2052', '90'), ('00', 'None', '전국', '2007', '508', '4771', '2309', '2098', '97'), ('00', 'None', '전국', '2008', '303', '4940', '2363', '2107', '95'), ('00', 'None', '전국', '2009', '300', '5462', '2501', '2194', '101'), ('00', 'None', '전국', '2010', '327', '5911', '2642', '2253'

In [8]:
# 구제시 + 구제시 아닌 곳 년도별로 csv 파일화하기

import _config
import pandas as pd
import _util

year = 2021
gis_year = 0
if year < 2005: gis_year = 2000
elif year < 2010: gis_year = 2005
elif year < 2015: gis_year = 2010
elif year < 2020: gis_year = 2015
elif year < 2023: gis_year = 2020
else: gis_year = 2023

file_nm = f'school_{year}'
sql = f'''
    do $$
    begin 
        if exists (select 1 from pg_matviews where matviewname = 'v_nogujesi_school_{year}') then 
        drop materialized view v_nogujesi_school_{year};
        end if;
        if not exists (select 1 from pg_matviews where matviewname = 'v_nogujesi_school_{year}') then
        create materialized view v_nogujesi_school_{year} as
        with sch as (
            select s.*
            from school s
            where "year" = {year}::text
        )
        , ngu as (
            select gs.sgg_cd, gs.sgg_nm, gs.geom
            from nogujaesi_sgg_{gis_year}_5179 gs
        )
        select row_number() over() as fid
        , s.*, n.geom
        from sch s
        inner join ngu n on n.sgg_cd = s.sgg_cd;
        end if;
    end $$;

    do $$
    begin
        if exists (select 1 from pg_matviews where matviewname = 'v_gujesi_school_{year}') then
        drop materialized view v_gujesi_school_{year};
        end if;
        if not exists (select 1 from pg_matviews where matviewname = 'v_gujesi_school_{year}') then
        create materialized view v_gujesi_school_{year} as
        with sch as (
            select s.*
            from school s
            where "year" = {year}::text
        )
        , gu as (
            select gs.sgg_cd, gs.sgg_nm, gs.geom
            from gujaesi_sgg_{gis_year}_5179 gs
        )
        select row_number() over() as fid
        , s.*, g.geom
        from sch s
        inner join gu g on g.sgg_cd = s.sgg_cd;
        end if;
    end $$;
'''
_util.execute_sql(sql)

cols = _util.getMatViewCols(f'v_gujesi_school_{year}')
# print('cols',cols)
# if cols is not None:
#     col_nms = [row[0] for row in cols]
#     _cols = list(col_nms)
#     cols = [f'\"{col}\"' for col in _cols if col != 'geom']

_col_list = [
    'sido_cd'
    , 'sgg_cd'
    , 'sgg_nm'
    , 'year'
    , '유치원'
    , '초등학교'
    , '중학교'
    , '고등학교'
    , '특수학교'
]

sql = f'''
    select {', '.join(_col_list)}
    from v_gujesi_school_{year} 
    union all 
    select {', '.join(_col_list)}
    from v_nogujesi_school_{year}
'''

# sql = f'''
#     do $$
#     begin
#         if exists (
#             select 1 
#             from pg_matviews 
#             where matviewname = 'v_gujesi_school_{year}'
#             ) 
#         and exists (
#             select 1 
#             from pg_matviews 
#             where matviewname = 'v_nogujesi_school_{year}'
#             ) 
#         then 
#             execute 'select * from v_gujesi_school_{year} union all select * from v_nogujesi_school_{year}';
#         end if;
#     end $$;
# '''
result = _util.execute_sql(sql)

_cols = {}
_col_list = [
    'sido_cd'
    , 'sgg_cd'
    , 'sgg_nm'
    , 'year'
    , '유치원'
    , '초등학교'
    , '중학교'
    , '고등학교'
    , '특수학교'
]
cols = [col for col in df.columns if col != 'Unnamed: 0']
for idx in range(len(cols)): _cols[idx] = _col_list[idx]
df = pd.DataFrame(list(result))
df.rename(columns=_cols, inplace=True)
df.to_csv(_config.school_write_path + '\\' + file_nm + '.csv', encoding=_config.euc_kr, index=False)

sql = f'''
    do $$
    begin 
        if exists (select 1 from pg_matviews where matviewname = 'v_gujesi_school_{year}') then
        drop materialized view v_gujesi_school_{year};
        end if;
        if exists (select 1 from pg_matviews where matviewname = 'v_nogujesi_school_{year}') then
        drop materialized view v_nogujesi_school_{year};
        end if;
    end $$;
'''
# _util.execute_sql(sql)





    do $$
    begin 
        if exists (select 1 from pg_matviews where matviewname = 'v_nogujesi_school_2021') then 
        drop materialized view v_nogujesi_school_2021;
        end if;
        if not exists (select 1 from pg_matviews where matviewname = 'v_nogujesi_school_2021') then
        create materialized view v_nogujesi_school_2021 as
        with sch as (
            select s.*
            from school s
            where "year" = 2021::text
        )
        , ngu as (
            select gs.sgg_cd, gs.sgg_nm, gs.geom
            from nogujaesi_sgg_2020_5179 gs
        )
        select row_number() over() as fid
        , s.*, n.geom
        from sch s
        inner join ngu n on n.sgg_cd = s.sgg_cd;
        end if;
    end $$;

    do $$
    begin
        if exists (select 1 from pg_matviews where matviewname = 'v_gujesi_school_2021') then
        drop materialized view v_gujesi_school_2021;
        end if;
        if not exists (select 1 from pg_matviews where matviewname